In [2]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os, glob
import gzip,tarfile

class ImageMaker:
    '''
    한 장의 사진으로 회전, 대칭 등 변환해서 여러장으로 만드는 클래스
    data_path_ :  원본 데이터 경로
    new_path_: 새롭게 저장할 경로
    title_:이미지 파일 이름
    '''

    # 객체 생성
    def __init__(self,data_path_,new_path_,title_):
        self.data_path=data_path_
        self.new_path = new_path_
        self.title = title_
        self.num_i = 1

    # 새로운 폴더 만들기
    def create_new_folder(self):
        if not os.path.exists(self.new_path):
            os.makedirs(self.new_path)
        else:pass

    # 이미지 크기 조절, 그레이 스케일, 누끼 등
    def resize_images(self,dsize_):
        images=glob.glob(self.data_path+'*')

        for image in images:
            org=cv2.imread(image,cv2.IMREAD_GRAYSCALE)  # 그레이스케일
            
            # 배경 지우기(누끼 따기)
            # org=remove(org)

            '''이미지 크기 조절
            - cv2.resize(src,이미지크기)
            - src : 원본이미지'''
            resized_=cv2.resize(org,dsize_)
            cv2.imwrite(self.new_path+f'{self.title}_{self.num_i}.png',resized_)
            self.num_i += 1


    # 이미지 회전
    def rotate_images(self,degrees_):
    # 회전 행렬 생성 : cv2.getRotationMatrix2D(중심축,각도,비율)
    # 회전 변환 행렬에 따른 회전 이미지 반환: cv2.warpAffine(src,회전 행렬,(w,h))
        images=glob.glob(self.new_path+'*')

        for image in images:
            img=cv2.imread(image)
            # 이미지 높이, 너비 채널 정보 추출 => 높이 , 너비 이용 회전 중심점 설정
            h,w,c=img.shape

            for degree in degrees_:
                matrix=cv2.getRotationMatrix2D((w/2,h/2),degree,0.9) #<= 비율
                rotate_=cv2.warpAffine(img,matrix,(w,h))
                cv2.imwrite(image.replace('.png',f'_rotate({degree}).png'),rotate_)
    
    # # 이미지 블러
    # def blur_images(self,blur_range):
    # # cv2.GaussianBlur(src,ksize,표준편차)
    # # ksize: 커널 크기. (가로, 세로) 튜플 (홀수만), 크기 커질수록 블러 강화
    #     images=glob.glob(self.new_path+'*')

    #     for image in images:
    #         img=cv2.imread(image)

    #         for b in blur_range:
    #             blur_=cv2.GaussianBlur(img,(b,b),0)
    #             cv2.imwrite(image.replace('.png',f'_blur({b}).png'),blur_)


    # 실행 !!!!!!!!!!!!!!!
    def run(self):
        self.create_new_folder()
        self.resize_images((28,28))  #<= 크기 
        degrees1=list(range(1,11,1))+list(range(350,360,1))
        self.rotate_images(degrees1)

# 폴더 불러오기
filepath='./HAND/'
files=glob.glob(filepath+'*')
files

for image_folder in files:
    data_path=image_folder+'/'
    new_path=f'{image_folder[:2]}new{image_folder[2:]}/'
    title=f'{image_folder[7:]}'
    name=title+'maker'
    name=ImageMaker(data_path, new_path, title)
    name.run()